In [413]:
import nrrd
import numpy as np
import pandas as pd
import json
import scipy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchio as tio
import plotly.graph_objects as go

raster_data, raster_header = nrrd.read('original/1_01-02-2018.nrrd')

voxel_vol = np.prod(raster_header["spacings"])
tbv = float(raster_header["tbv"])

crop_factor = 0.8
side_len = 96

voxel_vol, tbv

(0.08431986801139271, 181.52)

In [414]:
def get_avg_voxels(raster, tbv, voxel_vol):
    original_voxels = tbv / voxel_vol

    cropped_side_len = crop_raster(raster, return_shape_only=True, force_crop_factor=crop_factor)[0]
    resize_factor = side_len / cropped_side_len
    cropped_voxel_vol = voxel_vol / resize_factor
    
    cropped_voxels = tbv / cropped_voxel_vol

    print(f"Original voxels: {original_voxels}")
    print(f"Cropped voxels: {cropped_voxels}")

    return (original_voxels + cropped_voxels) / 2
    

def crop_raster(raster, return_shape_only=False, force_crop_factor=None):
    if force_crop_factor is None:
        crop_shape = (raster.shape * np.random.uniform(crop_factor, 1.0, size=3) * (1, np.random.uniform(crop_factor, 1.0), 1)).astype(int)
    else:
        crop_shape = (raster.shape * np.array([force_crop_factor, force_crop_factor**2, force_crop_factor])).astype(int)
    
    max_value = np.max(crop_shape)
    pad_shape = (max_value, max_value, max_value)
    
    print("Raster shape:", raster.shape)
    print("Crop shape:", crop_shape)
    print("Pad shape:", pad_shape)

    if return_shape_only: return pad_shape
    
    crop = tio.CropOrPad(target_shape=crop_shape)
    pad = tio.CropOrPad(target_shape=pad_shape)
    
    return pad(crop(np.expand_dims(raster, axis=0))).squeeze()

def prep_raster(raster, voxel_volume, new_side_len=100):
    # Crop the raster randomly and then pad it to be a cube. No deformation.
    original_raster = crop_raster(raster)
    original_side_len = original_raster.shape[0]
    original_voxel_vol = voxel_volume

    if np.array_equal(original_side_len, new_side_len):
        return original_raster, original_voxel_vol
    
    # Resize the raster to be a cube of side length new_side_len
    resize_factor = new_side_len / original_side_len

    new_raster = scipy.ndimage.zoom(original_raster, (resize_factor, resize_factor, resize_factor))
    new_voxel_vol = original_voxel_vol / resize_factor

    return new_raster, new_voxel_vol

norm_transform = tio.Compose([
    tio.transforms.RescaleIntensity(), 
    tio.transforms.ZNormalization()
])

print(get_avg_voxels(raster_data, tbv, voxel_vol))

Raster shape: (251, 217, 217)
Crop shape: [200 138 173]
Pad shape: (200, 200, 200)
Original voxels: 2152.7547929210978
Cropped voxels: 1033.322300602127
1593.0385467616125


In [415]:
prep_raster, prep_voxel_vol = prep_raster(raster_data, voxel_vol, new_side_len=side_len)

voxel_vol, prep_voxel_vol

Raster shape: (251, 217, 217)
Crop shape: [211 194 206]
Pad shape: (211, 211, 211)


(0.08431986801139271, 0.18532804323337357)

In [416]:
vol = scipy.ndimage.zoom(prep_raster, (0.3, 0.3, 0.3))
X, Y, Z = np.mgrid[:vol.shape[0], :vol.shape[1], :vol.shape[2]]

fig = go.Figure(data=go.Volume(
    x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
    value=vol.flatten(),
    isomin=50.0,
    isomax=255.0,
    opacity=0.2,
    surface_count=3,
    ))
fig.update_layout(scene_xaxis_showticklabels=False,
                scene_yaxis_showticklabels=False,
                scene_zaxis_showticklabels=False)
fig.show()

In [417]:
print("Original voxels:", tbv/voxel_vol)
print("Preprocessed voxels:", tbv/prep_voxel_vol)

Original voxels: 2152.7547929210978
Preprocessed voxels: 979.4524176323478
